In [5]:
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES
from Crypto.Protocol.KDF import scrypt
import os

# see https://nitratine.net/blog/post/python-gcm-encryption-tutorial/

class Encryptor:
    BUFFER_SIZE = 1024 * 1024  # The size in bytes that we read, encrypt and write to at once
    password = "password"
    SALT_LENGTH = 32
    NONCE_LENGTH = 16
    TAG_LENGTH = 16
    salt = get_random_bytes(SALT_LENGTH)
    key = scrypt(password, salt, key_len=32, N=2 ** 17, r=8, p=1)
    cipher = AES.new(key, AES.MODE_GCM)
    file_in = None
    file_out = None

    def __init__(self,input_filename = 't.txt', output_filename='t.txt.encrypted'):
        self.output_filename = output_filename
        self.input_filename = input_filename

    def generate_encrypted_filename(self):
        return self.input_filename + '.encrypted'

    def write_header(self, file_out):
        file_out.write(self.salt)
        file_out.write(self.cipher.nonce)

    def write_body(self, file_in, file_out):
        while len(data := file_in.read(self.BUFFER_SIZE)):
            encrypted_data = self.cipher.encrypt(data)
            file_out.write(encrypted_data)

    def write_footer(self, file_out):
        tag = self.cipher.digest()
        assert len(tag) == self.TAG_LENGTH
        file_out.write(tag)

    def do_encryption(self):
        with open(self.output_filename, 'wb') as file_out, \
             open(self.input_filename, 'rb') as file_in:
            self.write_header(file_out)
            self.write_body(file_in, file_out)
            self.write_footer(file_out)

    def do_decryption(self):
        with open(self.output_filename, 'wb') as file_out, \
             open(self.input_filename, 'rb') as file_in:
                self.read_file_in(file_in)
                self.read_and_output(file_in,file_out)
                self.verify(file_in)

    def read_file_in(self, file_in):
        self.salt = file_in.read(self.SALT_LENGTH)  # The salt we generated was 32 bits long
        nonce = file_in.read(self.NONCE_LENGTH)
        self.cipher = AES.new(self.key, AES.MODE_GCM, nonce=nonce)

    def read_and_output(self, file_in,file_out):
        file_in_size = os.path.getsize(self.input_filename)
        encrypted_data_size = file_in_size - self.SALT_LENGTH - self.NONCE_LENGTH - self.TAG_LENGTH  # Total - salt - nonce - tag = encrypted data
        # Read, decrypt and write out the data
        for chunk_size in RangeAndRemainder(encrypted_data_size,self.BUFFER_SIZE):
            data = file_in.read(chunk_size)
            decrypted_data = self.cipher.decrypt(data)
            file_out.write(decrypted_data)

    def verify(self,file_in):
        # Verify encrypted file is correct
        tag = file_in.read(16)
        self.cipher.verify(tag)

def RangeAndRemainder(whole, chunk):
    # yields equal chunks, then finishes with the remainder
    quot,remainder = divmod(whole,chunk)
    for _ in range(quot):
        yield chunk
    yield remainder



e = Encryptor(input_filename = 'requirements.txt', output_filename='t.txt.encrypted')
e.do_encryption()

e = Encryptor(input_filename = 't.txt.encrypted', output_filename='t3.txt')
e.do_decryption()

In [ ]:
import mmap

def mmap_io(filename):
    with open(filename, mode="r", encoding="utf8") as file_obj:
        with mmap.mmap(file_obj.fileno(), length=0, access=mmap.ACCESS_READ) as mmap_obj:
            text = mmap_obj.read()
            print(text)
mmap_io(filename='t.txt')